# Base Programme multilingues

    Import libraries Transformers pour modèles, gensim pour synonymes, Spacy et Stanza pour NLP
    l'importation des data se fait sur la base du Ficiher data préparation de Karim (à refaire)

In [1]:
import pandas as pd
import pke
import spacy
import torch
import stanza
import spacy_stanza
import warnings
import string
from gensim.models import KeyedVectors
import enchant    # Pour correction orthographique de synonymes
import numpy as np
import re
from transformers import pipeline
from transformers import AutoModel
from transformers import AutoModelForSequenceClassification
from transformers import AutoModelWithLMHead, AutoTokenizer
from tqdm.notebook import tqdm
from nltk.corpus import stopwords
tqdm.pandas()
warnings.filterwarnings("ignore")

In [2]:
# Changement pour l'utilisation de stanza
nlp_en = spacy.load("en_core_web_sm")
nlp_fr = spacy.load("fr_core_news_sm")
nlp_de = spacy.load("de_core_news_sm")
nlp_pl = spacy.load("pl_core_news_sm")
nlp_es = spacy.load("es_core_news_sm")

In [3]:
# Pas de stopwords en turc et en arabe sur spacy, nltk rien en polonais
stopWords = list(nlp_fr.Defaults.stop_words)
stopwords_fr = list(stopwords.words('french'))  
stopwords_fr = list(set(stopwords_fr + stopWords))
stopWords = list(nlp_en.Defaults.stop_words)
stopwords_en = list(stopwords.words('english'))  
stopwords_en = list(set(stopwords_en + stopWords))
stopWords = list(nlp_de.Defaults.stop_words)
stopwords_de = list(stopwords.words('german'))  
stopwords_de = list(set(stopwords_de + stopWords))
stopWords = list(nlp_es.Defaults.stop_words)
stopwords_es = list(stopwords.words('spanish'))  
stopwords_es = list(set(stopwords_es + stopWords))
stopwords_pl = list(nlp_pl.Defaults.stop_words)
stopwords_ar = list(stopwords.words('arabic'))
stopwords_tr = list(stopwords.words('turkish')) 
# print(len(stopwords_fr))
# print(len(stopwords_en))
# print(len(stopwords_de))
# print(len(stopwords_es))
# print(len(stopwords_pl))
# print(len(stopwords_ar))
# print(len(stopwords_tr))

In [4]:
#dico_langues = {'en':nlp_en,'fr':nlp_fr,'de':nlp_de,'es':nlp_es,'pl':nlp_pl}
langues = ['en','fr','es','de','pl','ar','tr']
stopwds_lg = {'en':stopwords_en,'fr':stopwords_fr,'es':stopwords_es,'de':stopwords_de,'pl':stopwords_pl,
              'ar':stopwords_ar,'tr':stopwords_tr}
dico_spacy = {'en':nlp_en,'fr':nlp_fr,'es':nlp_es,'de':nlp_de,'pl':nlp_pl}

**Importation des Data à mettre à jour et répartition par langues**

In [5]:
data = pd.read_csv('train_data_prep_v1.csv')

In [6]:
data

,pair_id,pair_lang,source_url_1,publish_date_1,source_url_2,publish_date_2,title_1,text_1,meta_description_1,meta_keywords_1,...,text_2,meta_description_2,meta_keywords_2,Geography,Entities,Time,Narrative,Overall,Style,Tone
0,1484084337_1484110209,en_en,https://www.washingtonpost.com,Wed Jan 1 00:00:00 2020,https://www.washingtonpost.com,Wed Jan 1 00:00:00 2020,Virginia man arrested in fatal DUI crash in We...,"MARTINSBURG, W.Va. — A suspected drunken drive...",Police in West Virginia say a suspected drunke...,"['Highway Fatal-DUI-West Virginia', 'Martinsbu...",...,"PORT-AU-PRINCE, Haiti — Haitian President Jove...",Haitian President Jovenel Moïse has broken wit...,"['CB-Haiti-Political Turmoil', 'Jean', 'Haiti'...",4.0,4.000000,1.000000,4.000000,4.000000,1.666667,2.000000
1,1484396422_1483924666,en_en,https://www.stlucianewsonline.com,Wed Jan 1 21:17:15 2020,https://www.thestar.com,Wed Jan 1 00:00:00 2020,Guyana: Three injured after car crashes into u...,Share This On:\n\nPin 11 Shares\n\n(NEWS ROOM ...,-,[''],...,BERLIN - A fire at a zoo in western Germany in...,BERLIN - A fire at a zoo in western Germany in...,"['smg2_world', 'smg_europe', 'smg2_news']",4.0,4.000000,1.000000,4.000000,3.666667,1.666667,1.333333
2,1484698254_1483758694,en_en,https://www.teaparty.org,NaN,https://www.timesofisrael.com,NaN,Trump Brings In 2020 At Mar-a-Lago: ‘We’re Goi...,(Breitbart) – President Donald Trump welcomed ...,NaN,[''],...,"PALM BEACH, United States — US President Donal...",US president says response to rioting by pro-I...,[''],1.0,2.000000,1.000000,2.333333,2.333333,1.000000,1.333333
3,1576314516_1576455088,en_en,https://gadgets.ndtv.com,NaN,https://gadgets.ndtv.com,NaN,Zomato Buys Uber's Food Delivery Business in I...,Uber has sold its online food-ordering busines...,Zomato on Tuesday announced it has acquired Ub...,['zomato uber eats business acquisition india ...,...,Rapid digitisation and growth in both online b...,The report by Google and Boston Consulting Gro...,['indian online food delivery market to hit us...,1.0,2.333333,2.666667,1.666667,2.000000,1.666667,1.666667
4,1484036253_1483894099,en_en,https://news.yahoo.com,Wed Jan 1 08:57:59 2020,NaN,NaN,"India approves third moon mission, months afte...",BENGALURU (Reuters) - India has approved its t...,India has approved its third lunar mission mon...,[''],...,NaN,NaN,NaN,1.0,1.250000,1.000000,1.250000,1.250000,1.000000,1.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4959,1586195445_1598778991,tr_tr,http://www.haberler.com,NaN,https://www.haberler.com,NaN,"BM, Aden'de 2 bini aşkın iç göçmenin selden za...","BM, Aden'de 2 bini aşkın iç göçmenin selden za...","Birleşmiş Milletler (BM), 2 bini aşkın iç göçm...","['Birleşmiş Milletler', 'Twitter', 'Yemen', 'G...",...,BM'den Yemen'de kadınların doğumda ölüm riski ...,BİRLEŞMİŞ Birleşmiş Milletler (BM) dünyanın en...,"['Birleşmiş Milletler', 'Yemen', 'Güncel', 'Ha...",1.0,2.000000,2.000000,4.000000,3.000000,1.000000,1.000000
4960,1590915424_1590940388,tr_tr,https://www.haberler.com,NaN,https://www.aksam.com.tr,NaN,Kovid-19'dan dolayı La Liga kulüplerinde hayat...,Kovid-19'dan dolayı La Liga kulüplerinde hayat...,Yeni tip koronavirüs (Kovid-19) salgınının eko...,"['Real Madrid', 'İspanya', 'La Liga', 'Futbol'...",...,Yeni tip koronavirüs (Kovid-19) salgınının eko...,Yeni tip koronavirüs (Kovid-19) salgınının eko...,"['İspanya 1. Futbol Ligi', 'la liga', 'koronav...",1.0,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000
4961,1526157103_1492737005,tr_tr,http://www.samanyoluhaber.com,Thu Feb 20 10:47:10 2020,https://www.fotomac.com.tr,Sun Jan 12 00:00:00 2020,Saray da çare olmadı: 'Borca boğulan dev kulüp...,\n\n\n\n\n\n\n\nİflas noktasındaki kulüplerin ...,SARAY DA ÇARE OLMADI: 'BORCA BOĞULAN DEV KULÜP...,"['Saray', 'da', 'çare', 'olmadı:', ""'Borca"", '...",...,"TFF, resmi internet sitesinden Beşiktaş'ın fai...","Federasyon, Başkan Çebi’nin yaptığı açıklamala...",[''],1.0,2.000000,3.0

In [7]:
data.pair_lang.value_counts()

en_en    1800
de_de     857
de_en     577
es_es     570
tr_tr     465
pl_pl     349
ar_ar     274
fr_fr      72
Name: pair_lang, dtype: int64

In [8]:
# remémorer numéro de ligne pour la suite - compléter les Nan : texte vide
data['ligne'] = data.index
data = data.fillna('')

_**Méthodologie : si 2 langues comme en_de : prendre traduction de "de" vers "en" (utiliser modele hugging face)**_

In [10]:
# séparation des datasets, le dernier étant à traduire en plus
anglais = data.loc[data.pair_lang == 'en_en',['ligne','title_1','title_2','text_1','text_2','Geography', 'Entities',
       'Time', 'Narrative', 'Overall', 'Style', 'Tone']].reset_index(drop=True)
allemand = data.loc[data.pair_lang == 'de_de',['ligne','title_1','title_2','text_1','text_2','Geography', 'Entities',
       'Time', 'Narrative', 'Overall', 'Style', 'Tone']].reset_index(drop=True)
espagnol = data.loc[data.pair_lang == 'es_es',['ligne','title_1','title_2','text_1','text_2','Geography', 'Entities',
       'Time', 'Narrative', 'Overall', 'Style', 'Tone']].reset_index(drop=True)
francais = data.loc[data.pair_lang == 'fr_fr',['ligne','title_1','title_2','text_1','text_2','Geography', 'Entities',
       'Time', 'Narrative', 'Overall', 'Style', 'Tone']].reset_index(drop=True)
polonais = data.loc[data.pair_lang == 'pl_pl',['ligne','title_1','title_2','text_1','text_2','Geography', 'Entities',
       'Time', 'Narrative', 'Overall', 'Style', 'Tone']].reset_index(drop=True)
arabe = data.loc[data.pair_lang == 'ar_ar',['ligne','title_1','title_2','text_1','text_2','Geography', 'Entities',
       'Time', 'Narrative', 'Overall', 'Style', 'Tone']].reset_index(drop=True)
turc = data.loc[data.pair_lang == 'tr_tr',['ligne','title_1','title_2','text_1','text_2','Geography', 'Entities',
       'Time', 'Narrative', 'Overall', 'Style', 'Tone']].reset_index(drop=True)
# all_ang = data.loc[data.pair_lang == 'de_en',['ligne','title_1','title_2','text_1','text_2','Geography', 'Entities',
#      'Time', 'Narrative', 'Overall', 'Style', 'Tone']].reset_index(drop=True)
# après traduction utiliser :
all_eng = pd.read_csv('allemand_anglais_traduit.csv')

In [11]:
# print(len(anglais))
# print(len(allemand))
# print(len(espagnol))
# print(len(francais))
# print(len(polonais))
# print(len(arabe))
# print(len(turc))
# print(len(all_eng))

**Fonctions Traduction**

In [12]:
# https://huggingface.co/models?language=de&pipeline_tag=translation&sort=downloads
# translation = pipeline('translation_en_to_de')  # en to de : t5-base ou helsinki en-de

In [13]:
def traduction_all_eng(df):
    # traduit le texte allemand en anglais mais on save le texte allemand 
    # texte original peut servir pour les entités - en allemand semble OK
    model = AutoModelWithLMHead.from_pretrained("Helsinki-NLP/opus-mt-de-en")
    tokenizer = AutoTokenizer.from_pretrained("Helsinki-NLP/opus-mt-de-en")
    translation = pipeline('translation_de_to_en', model=model, tokenizer=tokenizer)
    #df['title_1_de'] = df['title_1']; df['text_1_de'] = df['text_1']
    for i in tqdm(df.index.values):
        if len(df.title_1[i])>0: 
            df['title_1'][i] = translation(df.title_1[i])[0]['translation_text']
    
        trad = ''
        if len(df.text_1[i])>0:
            text1 = df.text_1[i].split('.')
        else:
            text1=[]
        for elt in text1:
            trad += translation(elt)[0]['translation_text']+'.'
        df['text_1'][i] = trad
    return df

In [14]:
# traduction réalisé
# traduction_all_eng(all_ang)
# all_ang.to_csv('allemand_anglais_traduit.csv')

**Essais de fonction Transformers en multilingues**

In [15]:
classifier = pipeline("sentiment-analysis")

No model was supplied, defaulted to distilbert-base-uncased-finetuned-sst-2-english (https://huggingface.co/distilbert-base-uncased-finetuned-sst-2-english)


In [16]:
print(classifier(anglais.title_1[15]))
print(classifier(anglais.title_2[15]))

[{'label': 'NEGATIVE', 'score': 0.9884806871414185}]
[{'label': 'POSITIVE', 'score': 0.9998032450675964}]


In [17]:
summarizer = pipeline("summarization", model="moussaKam/barthez-orangesum-title")

In [18]:
print(summarizer(francais.text_1[0]))
print(summarizer(francais.text_2[0]))

[{'summary_text': 'Immobilier : le cap du million de ventes dépassé en 2019'}]
[{'summary_text': 'Coronavirus : les soignants manquent toujours de masques'}]


In [19]:
print(summarizer(francais.text_1[3]))
print(summarizer(francais.text_2[3]))

[{'summary_text': 'Coronavirus : le chanteur et chanteur camerounais Manu Dibango est décédé'}]
[{'summary_text': "Coronavirus : l'Afrique pleure Manu Dibango victime du coronavirus"}]


In [20]:
from transformers import AutoModelForSequenceClassification
nli_model = AutoModelForSequenceClassification.from_pretrained("BaptisteDoyen/camembert-base-xnli")

In [21]:
tokenizer = AutoTokenizer.from_pretrained("BaptisteDoyen/camembert-base-xnli") 

In [22]:
# The idea is here to compute a probability of the form P(premise|hypothesis )P(premise∣hypothesis)
# sequences
premise = "le score pour les bleus est élevé"
hypothesis = "L'équipe de France a fait un bon match"
# tokenize and run through model
x = tokenizer.encode(premise, hypothesis, return_tensors='pt')
logits = nli_model(x)[0]
# we throw away "neutral" (dim 1) and take the probability of
# "entailment" (0) as the probability of the label being true 
entail_contradiction_logits = logits[:,::2]
probs = entail_contradiction_logits.softmax(dim=1)
prob_label_is_true = probs[:,0]
prob_label_is_true[0].tolist() * 100

86.40776872634888

In [24]:
premise = francais.title_1[3]
hypothesis = francais.title_2[3]
x = tokenizer.encode(premise, hypothesis, return_tensors='pt')
logits = nli_model(x)[0]
# we throw away "neutral" (dim 1) and take the probability of
# "entailment" (0) as the probability of the label being true 
entail_contradiction_logits = logits[:,::2]
probs = entail_contradiction_logits.softmax(dim=1)
prob_label_is_true = probs[:,0]
prob_label_is_true[0].tolist() * 100

69.10126805305481

In [25]:
# cette méthode est très intéressante .... index 0 : score 3.25, index 1 : 0.56, index 2 : 5.86, 4 : 0.53, 69 : 5.70, 67 : 2.52
# 68 : 2.02, 70 : 0.24, 71: 2.38
premise = francais.title_1[71]
hypothesis = francais.title_2[71]
x = tokenizer.encode(premise, hypothesis, return_tensors='pt')
logits = nli_model(x)[0]
# we throw away "neutral" (dim 1) and take the probability of
# "entailment" (0) as the probability of the label being true 
entail_contradiction_logits = logits[:,::2]
probs = entail_contradiction_logits.softmax(dim=1)
prob_label_is_true = probs[:,0]
prob_label_is_true[0].tolist() * 100

2.3809676989912987